# Measuring Fairness Metrics with the `Credit Card` Dataset

Return to the [castle](https://github.com/Nkluge-correa/teeny-tiny_castle).

Machine Learning Fairness is an area of research that focuses on ensuring that the use of machine learning algorithms is fair and unbiased, without reinforcing or amplifying existing societal inequalities. It is crucial to address this issue as machine learning models are increasingly being used in various fields, such as finance, employment, and criminal justice, where their decisions can have a significant impact on people's lives.

The importance of Machine Learning Fairness lies in the fact that algorithms can sometimes perpetuate or amplify biases that exist in the training data or the algorithm design. This can lead to discriminatory outcomes that disadvantage certain groups based on their race, gender, or other protected characteristics. Thus, ensuring fairness in machine learning algorithms is essential to build trust and accountability in their use.

<img src="https://miro.medium.com/max/1060/1*cc8OWxqKFXje4d_1eYrQkg.jpeg" width="600"/>

_Machine learning bias_, also sometimes called _algorithm bias or AI bias_ (NOT TO BE CONFUSED WITH A MATH BIAS), is a phenomenon that occurs when an algorithm produces results that are systemically prejudiced due to erroneous assumptions in the machine learning process.

The data set used is this notebook is the [`Credit Approval Data Set`](https://archive.ics.uci.edu/ml/datasets/credit+approval), made available by the [UCI Machine Learning Repository`](https://archive.ics.uci.edu/ml/index.php). This dataset contains labeled samples (approved/unapproved) of credit card applications.

Features of the `Credit Approval Data Set` have been masked to respect the anonymity of the subjects. However, we will infer the folliwing classes for the purpose of this notebook.

```python
features = ['Gender', 'Age', 'Debt', 'Married', 'Bank Client', 'Education',
'Race', 'Years Employed', 'Prior Default', 'Employed', 'Credit',
"Driver's License", 'Citizenship', 'Postal code', 'Income', 'Approval Status']
```

- A1 `Gender`: b, a.
- A2 `Age`: continuous.
- A3 `Debt`: continuous.
- A4 `Married`: u, y, l, t.
- A5 `Bank Client`: g, p, gg.
- A6 `Education`: c, d, cc, i, j, k, m, r, q, w, x, e, aa, ff.
- A7 `Race`: v, h, bb, j, n, z, dd, ff, o.
- A8 `Years Employed`: continuous.
- A9 `Prior Default`: t, f.
- A10 `Employed`: t, f.
- A11 `Credit`: continuous.
- A12 `Driver's License`: t, f.
- A13 `Citizenship`: g, p, s.
- A14 `Postal code`: continuous.
- A15 `Income`: continuous.
- A16 `Approval Status`: +,- 


Bellow we perform some simple pre-processing (e.g., substituting missing values for their `average value` or `mode`) to help the training of our model. Also, for simplicity, we will only retain 3 features for training the classifier, `Gender`,`Approval Status`, and `Prior Default`.

In [1]:
import numpy as np
import pandas as pd

df = pd.read_csv(r'data/dataCC.csv', header=0)
df.columns = ['Gender', 'Age', 'Debt', 'Married', 'Bank Client', 'Education',
                'Race', 'Years Employed', 'Prior Default', 'Employed', 'Credit',
                "Driver's License", 'Citizenship', 'Postal code', 'Income', 'Approval Status']

df = df.replace('?', np.nan)
df.fillna(df.mean(numeric_only= True), inplace=True)

for col in df.columns:
    if df[col].dtypes == 'object':
        df = df.fillna(df[col].value_counts().index[0])

df = df.drop(['Age', 'Married', 'Bank Client', 'Education',
                  'Race', 'Years Employed', 'Employed', 'Credit',
                  "Driver's License", 'Citizenship', 'Postal code', 'Income'], axis=1)

df['Gender'].replace({'b': 'Male', 'a': 'Female'}, inplace=True)
df['Approval Status'].replace({'+': 1, '-': 0}, inplace=True)
df['Prior Default'].replace({'t': 1, 'f': 0}, inplace=True)
display(df)

,Gender,Debt,Prior Default,Approval Status
0,Female,4.460,1,1
1,Female,0.500,1,1
2,Male,1.540,1,1
3,Male,5.625,1,1
4,Male,4.000,1,1
...,...,...,...,...
684,Male,10.085,0,0
685,Female,0.750,0,0
686,Female,13.500,0,0
687,Male,0.205,0,0


Now, let us try to se how our target (`Approval Status`) is related to the sensitive atribute of our dataset (`Gender`).

In [2]:
approved = []
not_approved = []

for element in list(df['Gender'].unique()):
    a = df[df['Gender'] == element]['Approval Status'].value_counts()[1]
    b = df[df['Gender'] == element]['Approval Status'].value_counts()[0]
    approved.append(a)
    not_approved.append(b)

import plotly.graph_objects as go

fig = go.Figure(data=[
    go.Bar(name='Approved', x=list(df['Gender'].unique()), y=approved),
    go.Bar(name='Not Approved', x=list(df['Gender'].unique()), y=not_approved)
])

fig.update_layout(
    barmode='group',
    template='plotly_dark',  
    xaxis_title="<b>Gender</b>",
    yaxis_title="<b>Approved Samples by Gender</b>",
    title='Distribution of <i>Approved Ratings</i> by "Gender"',
    paper_bgcolor='rgba(0, 0, 0, 0)',
    plot_bgcolor='rgba(0, 0, 0, 0)',
    )
fig.show()

Let us now see if Gender is correlated with `Approval Status`. If it is, this is already a signed that our future model could inherit this bias against a specific unprivileged class.

In [3]:

df_corr = df.copy()
df_corr['Gender'].replace({'Male': 1, 'Female': 0}, inplace=True)

import plotly.express as px

fig = px.imshow(df_corr.corr(numeric_only=True).values,
                labels=dict(x="Features", y="Features"),
                x=list(df_corr.columns),
                y=list(df_corr.columns),
                text_auto=True
                )
fig.update_xaxes(side='top')
fig.update_layout(template='plotly_dark',
                  title='Correlation Matrix',
                  coloraxis_showscale=False,
                  paper_bgcolor='rgba(0, 0, 0, 0)',
                  plot_bgcolor='rgba(0, 0, 0, 0)')
fig.show()

However, we can also speed up this vizualization process in our investigation by using tools like `pandas_profiling`.

<img src="https://warehouse-camo.ingress.cmh1.psfhosted.org/4c3692279382b860ef92ba7097363eefd6335d5a/68747470733a2f2f79646174612d70726f66696c696e672e79646174612e61692f646f63732f6173736574732f6c6f676f5f6865616465722e706e67" alt="drawing" width="600"/>

[`Pandas-profiling`](https://pypi.org/project/pandas-profiling/) generates profile reports from a pandas `DataFrame`. Extending a pandas `DataFrame` with `df.profile_report()`, will automatically generate a standardized univariate and multivariate report for data understanding.


In [ ]:
from pandas_profiling import ProfileReport

profile = ProfileReport(df, title="Pandas Profiling Report (Credit Card Dataset)") # generate report
profile.to_notebook_iframe() # Show report on notebook
profile.to_file("pandas_profiling_credit_card.html") # save report as an HTML page

According to the correlaton scores, Gender does not simes to be correlated with `Approval Status` (nor it could _legally_ be).

Now, let us write a simple `Logistic Regression` model so we can evaluate this algorithm against several fairness metrics.


In [4]:

from sklearn.metrics import confusion_matrix
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

seed = 42

train, test = train_test_split(df, stratify=df['Gender'], test_size=0.2, random_state=seed)

train.reset_index(drop=True, inplace=True)
test.reset_index(drop=True, inplace=True)

lr = LogisticRegression(random_state=seed)
lr.fit(train[['Debt', 'Prior Default']], train['Approval Status'])

score = lr.score(test[['Debt', 'Prior Default']], test['Approval Status'])
preds = lr.predict(test[['Debt', 'Prior Default']])

print(f'Accuracy: ' + '{:.2f}'.format(score * 100) + ' %')

matrix = confusion_matrix(test['Approval Status'], preds)

fig = px.imshow(matrix,
                labels=dict(x="Predicted", y="True label"),
                x=['Approved', 'Not approved'],
                y=['Approved', 'Not approved'],
                text_auto=True
                )
fig.update_xaxes(side='top')
fig.update_layout(template='plotly_dark',
                  title='Confusion Matrix',
                  coloraxis_showscale=False,
                  paper_bgcolor='rgba(0, 0, 0, 0)',
                  plot_bgcolor='rgba(0, 0, 0, 0)')
fig.show()


Accuracy: 87.68 %


The model seems quite balanced. However, how can we quantify how much this algorithm is "_fair_"? We can use `fairness metrics`.

### Statistical Fairness Metrics

_Fairness metrics are a set of measures that enable you to detect the presence of bias in your data or model._ For a full review of the most prominent definitions of fairness in algorithmic classification, we recommend [_Fairness Definitions Explained_](https://fairware.cs.umass.edu/papers/Verma.pdf). Below you will see how to apply these definitions in code.

Confusion Matrix cheatsheet:

![confusion-matrix](https://upload.wikimedia.org/wikipedia/commons/3/32/Binary_confusion_matrix.jpg)

- $TP$: True Positive
- $FP$: False Positive
- $TN$: True Neagative
- $FN$: False Negative

### Common Statistical Fairness Metrics

`Statistical Parity Ratio`: _Statistical Parity Ratio_ compares the proportion of members of a given group that were classified for the positive class (i.e., correctly or not, a.k.a., TP and FP) to another group (privileged $\times$ unprivileged).

$$\frac{Statistical\;Parity_{\;unprivileged}}{Statistical\;Parity_{\;privileged}} = \frac{(\frac{TP  +  FP}{TP  +  FP  +  TN  +  FN})^{unprivileged}}{(\frac{TP  +  FP}{TP  +  FP  +  TN  +  FN})^{privileged}}$$

`Equal Opportunity Ratio`: _Equal Opportunity ratio_ compares the true positive rate (i.e., TPR, a.k.a., _Sensitivity/Recall_) of different groups (privileged $\times$ unprivileged).

$$\frac{TPR_{\;unprivileged}}{TPR_{\;privileged}} = \frac{(\frac{TP}{TP+FN})^{unprivileged}}{(\frac{TP}{TP+FN})^{privileged}}$$

`Predictive Parity Ratio`: _Predictive Parity Ratio_ compares the positive predictive value (i.e., PPV, a.k.a., _Precision_) of different groups (privileged $\times$ unprivileged).

$$\frac{PPV_{\;unprivileged}}{PPV_{\;privileged}} = \frac{(\frac{TP}{TP+FP})^{unprivileged}}{(\frac{TP}{TP+FP})^{privileged}}$$

`Predictive Equality Ratio`: _Predictive Equality Ratio_ compares the false positive rate (FPR, a.k.a., _fall-out/false alarm ratio_) of different groups (privileged $\times$ unprivileged).

$$\frac{FPR_{\;unprivileged}}{FPR_{\;privileged}} = \frac{(\frac{FP}{FP+TN})^{unprivileged}}{(\frac{FP}{FP+TN})^{privileged}}$$

`Accuracy Equality Ratio`: _Accuracy Equality Ratio_ compares the proportion of members of a given group that were correctly classified (i.e., _accuracy_) to another group (privileged $\times$ unprivileged).

$$\frac{Accuracy_{\;unprivileged}}{Accuracy_{\;privileged}} = \frac{(\frac{TP  +  TN}{TP  +  FP  +  TN  +  FN})^{unprivileged}}{(\frac{TP  +  TN}{TP  +  FP  +  TN  +  FN})^{privileged}}$$

`Equalized Odds`: _Equalized Odds_ it is the most restrictive concept of ML Fairness. This criteria is only satisfied if both groups (privileged $\times$ unprivileged) have equal TPR and FPR.

$$\frac{TPR_{\;unprivileged}}{TPR_{\;privileged}} = 1 \;\land\;\frac{FPR_{\;unprivileged}}{FPR_{\;privileged}} = 1$$

Let us know to implement a function that calculates all these metrics.

In [5]:

def calc_fair(model, DataFrame, protected_atributte, group_priv, group_unpriv, label):
    """
    The function calc_fair computes several fairness metrics for a given machine 
    learning model on a test set DataFrame. The fairness metrics calculated include 
    statistical parity ratio, true positive rate, positive predictive value, 
    false positive rate, accuracy, equal opportunity ratio, predictive parity 
    ratio, predictive equality ratio, and accuracy equality ratio. The function 
    takes in the following arguments:

    Args:
    --------
        - model: The trained machine learning model to evaluate fairness on.
        - DataFrame: The test set data used to evaluate the model.
        - protected_attribute: The name of the protected attribute in the DataFrame.
        - group_priv: The value of the protected attribute for the privileged group.
        - group_unpriv: The value of the protected attribute for the unprivileged group.
        - label: The name of the column in the DataFrame that contains the ground truth labels.
    
    Returns:
    --------
    The function returns a dictionary containing the fairness metric names and their 
    corresponding scores. The scores are rounded to two decimal places. Additionally, 
    the function returns the equalized odds as a string.
    """
    test_set = DataFrame.copy()

    test_set_priv_labels, test_set_priv = list(test_set[test_set[protected_atributte] == group_priv][label]), test_set[test_set[protected_atributte] == group_priv].drop([label, protected_atributte], axis = 1)
    test_set_unpriv_labels, test_set_unpriv = list(test_set[test_set[protected_atributte] == group_unpriv][label]), test_set[test_set[protected_atributte] == group_unpriv].drop([label, protected_atributte], axis = 1)
    
    preds_priv = model.predict(test_set_priv)
    preds_unpriv = model.predict(test_set_unpriv)

    TN_PV, FP_PV, FN_PV, TP_PV = confusion_matrix(test_set_priv_labels, preds_priv).ravel()
    TN_UPV, FP_UPV, FN_UPV, TP_UPV = confusion_matrix(test_set_unpriv_labels, preds_unpriv).ravel()

    statistical_parity_priv = (TP_PV + FP_PV)/(TP_PV + FP_PV + TN_PV + FN_PV)  # STATISTICAL PARITY RATIO
    statistical_parity_unpriv = (TP_UPV + FP_UPV)/(TP_UPV + FP_UPV + TN_UPV + FN_UPV)  # STATISTICAL PARITY RATIO
    equal_oportunity_priv = TP_PV / (TP_PV+FN_PV)  # TRUE POSITIVE RATIO
    equal_oportunity_unpriv = TP_UPV / (TP_UPV+FN_UPV)  # TRUE POSITIVE RATIO
    predictive_parity_priv = TP_PV/(TP_PV + FP_PV)  # POSITIVE PREDICTIVE VALUE
    predictive_parity_unpriv = TP_UPV/(TP_UPV + FP_UPV)  # POSITIVE PREDICTIVE VALUE
    predictive_equality_priv = FP_PV / (FP_PV+TN_PV)  # FALSE POSITIVE RATE
    predictive_equality_unpriv = FP_UPV / (FP_UPV+TN_UPV)  # FALSE POSITIVE RATE
    accuracy_equality_priv = (TP_PV + TN_PV)/(TP_PV + FP_PV + TN_PV + FN_PV)  # ACCURACY EQUALITY RATIO
    accuracy_equality_unpriv = (TP_UPV + TN_UPV)/(TP_UPV + FP_UPV + TN_UPV + FN_UPV)  # ACCURACY EQUALITY RATIO 

    if statistical_parity_priv >= statistical_parity_unpriv:
        statistical_parity_ratio = statistical_parity_unpriv/statistical_parity_priv
    elif statistical_parity_priv < statistical_parity_unpriv:
        statistical_parity_ratio = statistical_parity_priv/statistical_parity_unpriv

    if equal_oportunity_priv >= equal_oportunity_unpriv:
        equal_oportunity_ratio = equal_oportunity_unpriv/equal_oportunity_priv
    elif equal_oportunity_priv < equal_oportunity_unpriv:
        equal_oportunity_ratio = equal_oportunity_priv/equal_oportunity_unpriv
    
    if predictive_parity_priv >= predictive_parity_unpriv:
        predictive_parity_ratio = predictive_parity_unpriv/predictive_parity_priv
    elif predictive_parity_priv < predictive_parity_unpriv:
        predictive_parity_ratio = predictive_parity_priv/predictive_parity_unpriv

    if predictive_equality_priv >= predictive_equality_unpriv:
        predictive_equality_ratio = predictive_equality_unpriv/predictive_equality_priv
    elif predictive_equality_priv < predictive_equality_unpriv:
        predictive_equality_ratio = predictive_equality_priv/predictive_equality_unpriv
    
    if accuracy_equality_priv >= accuracy_equality_unpriv:
        accuracy_equality_ratio = accuracy_equality_unpriv/accuracy_equality_priv
    elif accuracy_equality_priv < accuracy_equality_unpriv:
        accuracy_equality_ratio = accuracy_equality_priv/accuracy_equality_unpriv

    data = {'Fairness Metrics': ['Chance of receiving the positive class - privileged',
                                'Chance of receiving the positive class - unprivileged',
                                'Statistical Parity Ratio (SPR)',
                                'True Positive Rate - privileged',
                                'True Positive Rate - unprivileged',
                                'Equal Opportunity Ratio (EOR)',
                                'Positive Predictive Value - privileged',
                                'Positive Predictive Value - unprivileged',
                                'Predictive Parity Ratio (PPR)',
                                'False Positive Rate - privileged',
                                'False Positive Rate - unprivileged',
                                'Predictive Equality Ratio (PER)',
                                'Accuracy - privileged',
                                'Accuracy - unprivileged',
                                'Accuracy Equality Ratio (AER)',
                                'Equalized Odds'],
            'Scores': [round(statistical_parity_priv, 2),
                        round(statistical_parity_unpriv, 2),
                        round(statistical_parity_ratio,2),
                        round(equal_oportunity_priv, 2),
                        round(equal_oportunity_unpriv, 2),
                        round(equal_oportunity_ratio, 2),
                        round(predictive_parity_priv,2),
                        round(predictive_parity_unpriv,2),
                        round(predictive_parity_ratio,2),
                        round(predictive_equality_priv,2),
                        round(predictive_equality_unpriv,2),
                        round(predictive_equality_ratio,2),
                        round(accuracy_equality_priv,2),
                        round(accuracy_equality_unpriv,2),
                        round(accuracy_equality_ratio,2),
                        f'TPR: {round(equal_oportunity_priv, 2)} vs {round(equal_oportunity_unpriv, 2)}. FPR: {round(predictive_equality_priv,2)} vs {round(predictive_equality_unpriv,2)}']
            }
    return pd.DataFrame(data).set_index('Fairness Metrics')

from IPython.display import Markdown

fairness_df = calc_fair(lr, test, 'Gender', 'Female', 'Male', 'Approval Status')
display(Markdown(fairness_df.to_markdown()))

| Fairness Metrics                                      | Scores                               |
|:------------------------------------------------------|:-------------------------------------|
| Chance of receiving the positive class - privileged   | 0.5                                  |
| Chance of receiving the positive class - unprivileged | 0.5                                  |
| Statistical Parity Ratio (SPR)                        | 1.0                                  |
| True Positive Rate - privileged                       | 0.86                                 |
| True Positive Rate - unprivileged                     | 0.89                                 |
| Equal Opportunity Ratio (EOR)                         | 0.96                                 |
| Positive Predictive Value - privileged                | 0.86                                 |
| Positive Predictive Value - unprivileged              | 0.88                                 |
| Predictive Parity Ratio (PPR)                         | 0.98                                 |
| False Positive Rate - privileged                      | 0.14                                 |
| False Positive Rate - unprivileged                    | 0.12                                 |
| Predictive Equality Ratio (PER)                       | 0.86                                 |
| Accuracy - privileged                                 | 0.86                                 |
| Accuracy - unprivileged                               | 0.89                                 |
| Accuracy Equality Ratio (AER)                         | 0.97                                 |
| Equalized Odds                                        | TPR: 0.86 vs 0.89. FPR: 0.14 vs 0.12 |

Results show that, according to most fairness metrics, this model is unbiased to a tolerable degree.

Fairness metrics can help us find out if our model is discriminating according to any particular definition of fairness. However, as an [impossibility theorem](https://arxiv.org/abs/2007.06024), some Fairness metrics (_Statistical Parity_, _Equalized Odds_, _Predictive Parity_) are incompatible, and cannot be completely satisfied simultaneously. Thus, the choice of which metric to use must be made according to the context of an application (i.e., benefit awarding, medical diagnosis, etc.).

---

Return to the [castle](https://github.com/Nkluge-correa/teeny-tiny_castle).
